In [1]:
import os
import random
from transformers import RobertaTokenizer, T5ForConditionalGeneration, T5Tokenizer, T5Model, T5Config
import sys
sys.path.append('../')
from data.dataset import CodeSnifferDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sniffer import CodeSnifferNetwork
import torch
import torch.optim as optim
from train import train_model

random.seed(90)

In [ ]:
train_percent = 0.8
batch_size = 64
lr = 0.0001
num_labels = 8
num_epochs = 3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#codeModel.config.n_positions = 4096
#codeTokenizer.model_max_length = 4096

In [ ]:
dataset = CodeSnifferDataset(annotations_file="../data/filtered_data.csv", code_files_dir="../data/code_files")

train_size = int(train_percent * len(dataset))    # 80% for training and 20% for testing.
test_size = len(dataset) - train_size
training_data, test_data = random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

dataloaders = {"train": train_dataloader, "val": test_dataloader}

In [ ]:
codeModel = CodeSnifferNetwork(num_labels=num_labels)
# codeModel = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')

criterion = torch.nn.BCELoss()
optimizer = optim.Adam(codeModel.parameters(), lr=lr)

In [ ]:
train_model(codeModel, dataloaders, criterion, optimizer, device, num_epochs)